# Deep Neural Network for MNIST Classification

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\user\tensorflow_datasets\mnist\3.0.1.incomplete5P514Y\mnist-train.tfrecord*...:   0%|      …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\user\tensorflow_datasets\mnist\3.0.1.incomplete5P514Y\mnist-test.tfrecord*...:   0%|       …

Dataset mnist downloaded and prepared to C:\Users\user\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


## Preprocessing Data

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))
test_inputs, test_targets = next(iter(test_data))

## Model

### Outline the model

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 300

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

In [5]:
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0008)
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [7]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 7s - loss: 0.0504 - accuracy: 0.9856 - val_loss: 0.0478 - val_accuracy: 0.9882 - 7s/epoch - 13ms/step
Epoch 2/5
540/540 - 7s - loss: 0.0452 - accuracy: 0.9873 - val_loss: 0.0574 - val_accuracy: 0.9830 - 7s/epoch - 13ms/step
Epoch 3/5
540/540 - 7s - loss: 0.0419 - accuracy: 0.9882 - val_loss: 0.0501 - val_accuracy: 0.9855 - 7s/epoch - 13ms/step
Epoch 4/5
540/540 - 7s - loss: 0.0360 - accuracy: 0.9900 - val_loss: 0.0416 - val_accuracy: 0.9882 - 7s/epoch - 12ms/step
Epoch 5/5
540/540 - 8s - loss: 0.0350 - accuracy: 0.9898 - val_loss: 0.0351 - val_accuracy: 0.9908 - 8s/epoch - 15ms/step


## Test the model

In [8]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 711ms/step - loss: 0.0922 - accuracy: 0.9809


In [9]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 98.09%
